In [1]:
import torch
from torch import Tensor
import prologue as prologue
import dlc_practical_4_embryo as pract_4
import time 
import math

from torch import nn
from torch.nn import functional as F

* Using MNIST
Using downloaded and verified file: ./data/mnist/raw/train-images-idx3-ubyte.gz
Extracting ./data/mnist/raw/train-images-idx3-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/raw/train-labels-idx1-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/t10k-labels-idx1-ubyte.gz
Extracting ./data/mnist/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/raw/
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
0 0.9033293500542641
1 0.7755392417311668
2 0.6989829242229462
3 0.6369436867535114
4 0.585752759128809
5 0.5477792844176292
6 0.5352976620197296
7 0.49286913499236107
8 0.4682560861110687
9 0.45754126086831093
10 0.41976265981793404
11 0.4122622720897198
12 0.3939913362

In [2]:
def sigma(x):
    return x.tanh()

def dsigma(x):
    return 4 * (x.exp() + x.mul(-1).exp()).pow(-2)

######################################################################

def loss(v, t):
    return (v - t).pow(2).sum()

def dloss(v, t):
    return 2 * (v - t)

######################################################################

def forward_pass(w1, b1, w2, b2, x):
    x0 = x
    s1 = w1.mv(x0) + b1
    x1 = sigma(s1)
    s2 = w2.mv(x1) + b2
    x2 = sigma(s2)

    return x0, s1, x1, s2, x2

def backward_pass(w1, b1, w2, b2,
                  t,
                  x, s1, x1, s2, x2,
                  dl_dw1, dl_db1, dl_dw2, dl_db2):
    x0 = x
    dl_dx2 = dloss(x2, t)
    dl_ds2 = dsigma(s2) * dl_dx2
    dl_dx1 = w2.t().mv(dl_ds2)
    dl_ds1 = dsigma(s1) * dl_dx1

    dl_dw2.add_(dl_ds2.view(-1, 1).mm(x1.view(1, -1)))
    dl_db2.add_(dl_ds2)
    dl_dw1.add_(dl_ds1.view(-1, 1).mm(x0.view(1, -1)))
    dl_db1.add_(dl_ds1)



## Write the code to train and test a MLP with one hidden layer of 50 units



In [3]:
# Load the data using the provided prologue.load ̇data function, with one-hot label vectors and normalized inputs. 
# Multiply the target label vectors by ζ = 0.9 (so that they are strictly in the value range of tanh).
# Create the four weight and bias tensors, and fill them with random values sampled according to 𝒩 (0, ε) with ε = 1e − 6.
# Create the four tensors to sum up the gradients on individual samples, with respect to the weights and biases.
# Perform 1, 000 gradient steps with a step size η equal to 0.1 divided by the number of training samples.
# Each of these steps requires to reset to zero the tensors for summing up the gradients, and doing a forward and a backward pass for each training example.
# Compute and print the training loss, training error and test error after every step using the class of maximum response as the predicted one.

train_input, train_target, test_input, test_target = prologue.load_data(one_hot_labels = True,
                                                                        normalize = True)

nb_classes = train_target.size(1)
nb_train_samples = train_input.size(0)

zeta = 0.90

train_target = train_target * zeta
test_target = test_target * zeta

nb_hidden = 50
eta = 1e-1 / nb_train_samples
epsilon = 1e-6

w1 = torch.empty(nb_hidden, train_input.size(1)).normal_(0, epsilon)
b1 = torch.empty(nb_hidden).normal_(0, epsilon)
w2 = torch.empty(nb_classes, nb_hidden).normal_(0, epsilon)
b2 = torch.empty(nb_classes).normal_(0, epsilon)

dl_dw1 = torch.empty(w1.size())
dl_db1 = torch.empty(b1.size())
dl_dw2 = torch.empty(w2.size())
dl_db2 = torch.empty(b2.size())

for k in range(1000):

    # Back-prop

    acc_loss = 0
    nb_train_errors = 0

    dl_dw1.zero_()
    dl_db1.zero_()
    dl_dw2.zero_()
    dl_db2.zero_()

    for n in range(nb_train_samples):
        x0, s1, x1, s2, x2 = forward_pass(w1, b1, w2, b2, train_input[n])

        pred = x2.max(0)[1].item()
        if train_target[n, pred] < 0.5: nb_train_errors = nb_train_errors + 1
        acc_loss = acc_loss + loss(x2, train_target[n])

        backward_pass(w1, b1, w2, b2,
                      train_target[n],
                      x0, s1, x1, s2, x2,
                      dl_dw1, dl_db1, dl_dw2, dl_db2)

    # Gradient step

    w1 = w1 - eta * dl_dw1
    b1 = b1 - eta * dl_db1
    w2 = w2 - eta * dl_dw2
    b2 = b2 - eta * dl_db2

    # Test error

    nb_test_errors = 0

    for n in range(test_input.size(0)):
        _, _, _, _, x2 = forward_pass(w1, b1, w2, b2, test_input[n])

        pred = x2.max(0)[1].item()
        if test_target[n, pred] < 0.5: nb_test_errors = nb_test_errors + 1

    print('{:d} acc_train_loss {:.02f} acc_train_error {:.02f}% test_error {:.02f}%'
          .format(k,
                  acc_loss,
                  (100 * nb_train_errors) / train_input.size(0),
                  (100 * nb_test_errors) / test_input.size(0)))

* Using MNIST
Using downloaded and verified file: ./data/mnist/raw/train-images-idx3-ubyte.gz
Extracting ./data/mnist/raw/train-images-idx3-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/raw/train-labels-idx1-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/t10k-labels-idx1-ubyte.gz
Extracting ./data/mnist/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/raw/
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
0 acc_train_loss 810.00 acc_train_error 90.00% test_error 90.10%
1 acc_train_loss 780.58 acc_train_error 88.30% test_error 90.10%
2 acc_train_loss 761.77 acc_train_error 88.30% test_error 90.10%
3 acc_train_loss 749.74 acc_train_error 88.30% test_error 90.10%
4 acc_train

114 acc_train_loss 291.94 acc_train_error 11.50% test_error 20.70%
115 acc_train_loss 290.91 acc_train_error 11.40% test_error 21.70%
116 acc_train_loss 291.37 acc_train_error 11.90% test_error 20.50%
117 acc_train_loss 293.56 acc_train_error 11.10% test_error 21.60%
118 acc_train_loss 297.64 acc_train_error 12.30% test_error 20.50%
119 acc_train_loss 301.61 acc_train_error 11.80% test_error 22.30%
120 acc_train_loss 303.21 acc_train_error 12.60% test_error 20.50%
121 acc_train_loss 299.53 acc_train_error 10.90% test_error 22.90%
122 acc_train_loss 294.02 acc_train_error 12.90% test_error 20.30%
123 acc_train_loss 291.61 acc_train_error 10.00% test_error 23.80%
124 acc_train_loss 294.83 acc_train_error 13.80% test_error 22.50%
125 acc_train_loss 308.68 acc_train_error 11.00% test_error 26.30%
126 acc_train_loss 323.23 acc_train_error 15.90% test_error 24.10%
127 acc_train_loss 335.36 acc_train_error 12.70% test_error 25.40%
128 acc_train_loss 318.27 acc_train_error 15.50% test_error 21

238 acc_train_loss 179.34 acc_train_error 4.70% test_error 15.50%
239 acc_train_loss 176.66 acc_train_error 5.20% test_error 16.20%
240 acc_train_loss 173.92 acc_train_error 4.50% test_error 15.40%
241 acc_train_loss 171.81 acc_train_error 5.20% test_error 16.20%
242 acc_train_loss 170.27 acc_train_error 4.60% test_error 15.50%
243 acc_train_loss 169.08 acc_train_error 5.40% test_error 16.50%
244 acc_train_loss 168.81 acc_train_error 4.60% test_error 14.90%
245 acc_train_loss 168.78 acc_train_error 5.20% test_error 16.90%
246 acc_train_loss 170.14 acc_train_error 4.50% test_error 14.80%
247 acc_train_loss 171.57 acc_train_error 5.30% test_error 16.80%
248 acc_train_loss 174.80 acc_train_error 4.60% test_error 15.00%
249 acc_train_loss 177.28 acc_train_error 4.90% test_error 17.00%
250 acc_train_loss 180.75 acc_train_error 4.30% test_error 15.10%
251 acc_train_loss 182.52 acc_train_error 4.80% test_error 17.30%
252 acc_train_loss 183.62 acc_train_error 4.50% test_error 15.70%
253 acc_tr

364 acc_train_loss 154.22 acc_train_error 3.60% test_error 18.70%
365 acc_train_loss 150.32 acc_train_error 4.00% test_error 16.00%
366 acc_train_loss 145.52 acc_train_error 3.10% test_error 17.00%
367 acc_train_loss 142.93 acc_train_error 3.50% test_error 16.30%
368 acc_train_loss 143.33 acc_train_error 2.90% test_error 15.90%
369 acc_train_loss 144.31 acc_train_error 3.10% test_error 16.30%
370 acc_train_loss 146.62 acc_train_error 2.80% test_error 15.30%
371 acc_train_loss 145.56 acc_train_error 3.00% test_error 15.80%
372 acc_train_loss 142.20 acc_train_error 2.70% test_error 15.00%
373 acc_train_loss 135.90 acc_train_error 3.00% test_error 16.00%
374 acc_train_loss 129.80 acc_train_error 2.70% test_error 14.60%
375 acc_train_loss 124.73 acc_train_error 2.90% test_error 16.20%
376 acc_train_loss 122.00 acc_train_error 2.80% test_error 14.40%
377 acc_train_loss 120.53 acc_train_error 2.90% test_error 16.60%
378 acc_train_loss 121.44 acc_train_error 2.70% test_error 15.40%
379 acc_tr

490 acc_train_loss 101.30 acc_train_error 1.70% test_error 16.50%
491 acc_train_loss 102.01 acc_train_error 1.30% test_error 14.60%
492 acc_train_loss 102.08 acc_train_error 1.60% test_error 16.00%
493 acc_train_loss 103.01 acc_train_error 1.40% test_error 14.50%
494 acc_train_loss 102.91 acc_train_error 1.60% test_error 15.60%
495 acc_train_loss 103.55 acc_train_error 1.40% test_error 14.40%
496 acc_train_loss 102.97 acc_train_error 1.70% test_error 15.50%
497 acc_train_loss 103.11 acc_train_error 1.50% test_error 14.50%
498 acc_train_loss 102.15 acc_train_error 1.70% test_error 15.90%
499 acc_train_loss 101.99 acc_train_error 1.50% test_error 14.50%
500 acc_train_loss 101.09 acc_train_error 1.70% test_error 15.80%
501 acc_train_loss 101.07 acc_train_error 1.30% test_error 15.00%
502 acc_train_loss 100.67 acc_train_error 1.60% test_error 16.10%
503 acc_train_loss 101.06 acc_train_error 1.20% test_error 14.80%
504 acc_train_loss 101.19 acc_train_error 1.50% test_error 16.00%
505 acc_tr

616 acc_train_loss 108.99 acc_train_error 1.10% test_error 16.20%
617 acc_train_loss 113.18 acc_train_error 0.90% test_error 15.00%
618 acc_train_loss 113.59 acc_train_error 1.20% test_error 16.40%
619 acc_train_loss 113.37 acc_train_error 0.90% test_error 15.10%
620 acc_train_loss 107.62 acc_train_error 1.30% test_error 16.30%
621 acc_train_loss 101.81 acc_train_error 1.20% test_error 14.80%
622 acc_train_loss 94.85 acc_train_error 1.10% test_error 15.50%
623 acc_train_loss 89.30 acc_train_error 1.00% test_error 14.50%
624 acc_train_loss 85.02 acc_train_error 1.00% test_error 15.10%
625 acc_train_loss 81.86 acc_train_error 1.00% test_error 14.40%
626 acc_train_loss 79.72 acc_train_error 0.90% test_error 15.30%
627 acc_train_loss 78.12 acc_train_error 1.00% test_error 14.80%
628 acc_train_loss 77.06 acc_train_error 0.90% test_error 15.50%
629 acc_train_loss 76.21 acc_train_error 0.90% test_error 14.60%
630 acc_train_loss 75.64 acc_train_error 1.00% test_error 15.20%
631 acc_train_loss 

742 acc_train_loss 85.04 acc_train_error 0.80% test_error 14.00%
743 acc_train_loss 82.80 acc_train_error 0.80% test_error 16.70%
744 acc_train_loss 80.26 acc_train_error 0.80% test_error 14.00%
745 acc_train_loss 77.67 acc_train_error 0.80% test_error 16.20%
746 acc_train_loss 76.43 acc_train_error 0.80% test_error 14.20%
747 acc_train_loss 75.80 acc_train_error 0.80% test_error 16.40%
748 acc_train_loss 77.05 acc_train_error 0.80% test_error 14.50%
749 acc_train_loss 78.60 acc_train_error 0.80% test_error 16.70%
750 acc_train_loss 81.99 acc_train_error 0.80% test_error 14.80%
751 acc_train_loss 84.42 acc_train_error 0.70% test_error 16.70%
752 acc_train_loss 87.79 acc_train_error 0.80% test_error 15.00%
753 acc_train_loss 88.34 acc_train_error 0.90% test_error 16.60%
754 acc_train_loss 88.56 acc_train_error 0.80% test_error 14.80%
755 acc_train_loss 85.96 acc_train_error 0.90% test_error 16.40%
756 acc_train_loss 83.04 acc_train_error 0.80% test_error 14.50%
757 acc_train_loss 79.58 

870 acc_train_loss 64.34 acc_train_error 0.60% test_error 16.10%
871 acc_train_loss 66.37 acc_train_error 0.70% test_error 14.00%
872 acc_train_loss 68.70 acc_train_error 0.70% test_error 15.70%
873 acc_train_loss 71.67 acc_train_error 0.70% test_error 14.40%
874 acc_train_loss 74.00 acc_train_error 0.70% test_error 16.30%
875 acc_train_loss 76.34 acc_train_error 0.70% test_error 14.30%
876 acc_train_loss 76.90 acc_train_error 0.70% test_error 16.10%
877 acc_train_loss 77.09 acc_train_error 0.70% test_error 14.20%
878 acc_train_loss 75.53 acc_train_error 0.70% test_error 16.00%
879 acc_train_loss 74.02 acc_train_error 0.60% test_error 14.10%
880 acc_train_loss 71.70 acc_train_error 0.60% test_error 16.40%
881 acc_train_loss 69.84 acc_train_error 0.50% test_error 14.10%
882 acc_train_loss 67.85 acc_train_error 0.60% test_error 16.30%
883 acc_train_loss 66.29 acc_train_error 0.50% test_error 14.00%
884 acc_train_loss 64.83 acc_train_error 0.60% test_error 16.00%
885 acc_train_loss 63.59 

998 acc_train_loss 55.59 acc_train_error 0.50% test_error 14.80%
999 acc_train_loss 55.79 acc_train_error 0.50% test_error 15.10%


In [4]:

train_input, train_target, test_input, test_target = \
    prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)

######################################################################

class Net(nn.Module):
    def __init__(self, nb_hidden):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

######################################################################

class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        nb_hidden = 200
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=2)
        self.fc1 = nn.Linear(9 * 64, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(-1, 9 * 64)))
        x = self.fc2(x)
        return x

######################################################################

def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 100):
    criterion = nn.MSELoss()
    eta = 1e-1

    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            acc_loss = acc_loss + loss.item()

            model.zero_grad()
            loss.backward()

            with torch.no_grad():
                for p in model.parameters():
                    p -= eta * p.grad

        print(e, acc_loss)

def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if target[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors

######################################################################

mini_batch_size = 100

* Using MNIST
Using downloaded and verified file: ./data/mnist/raw/train-images-idx3-ubyte.gz
Extracting ./data/mnist/raw/train-images-idx3-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/raw/train-labels-idx1-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/raw/
Using downloaded and verified file: ./data/mnist/raw/t10k-labels-idx1-ubyte.gz
Extracting ./data/mnist/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/raw/
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [5]:
for nh in [ 10, 50, 200, 500, 2500 ]:
    model = Net(nh)
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net nh={:d} {:0.2f}%% {:d}/{:d}'.format(nh,
                                                              (100 * nb_test_errors) / test_input.size(0),
                                                              nb_test_errors, test_input.size(0)))

0 1.1391609534621239
1 0.9051785245537758
2 0.8221799284219742
3 0.7653302624821663
4 0.7241864055395126
5 0.6990223005414009
6 0.6750767603516579
7 0.6460570767521858
8 0.6449301950633526
9 0.6402152329683304
10 0.6068916469812393
11 0.5912947244942188
12 0.5791573040187359
13 0.5672611594200134
14 0.5574028864502907
15 0.5484873093664646
16 0.5387048870325089
17 0.5293684490025043
18 0.52151720225811
19 0.5152613110840321
20 0.5096364505589008
21 0.5038684867322445
22 0.49711383506655693
23 0.4882248528301716
24 0.4817071333527565
25 0.4765251539647579
26 0.4722793698310852
27 0.48137811571359634
28 0.47734659165143967
29 0.4582723006606102
30 0.45610784739255905
31 0.45185163989663124
32 0.4425029791891575
33 0.4361945353448391
34 0.44806062430143356
35 0.4385612867772579
36 0.426317285746336
37 0.4233347624540329
38 0.42176126316189766
39 0.4124268926680088
40 0.407599862664938
41 0.4142635762691498
42 0.4066314361989498
43 0.4002796672284603
44 0.39483505487442017
45 0.39072086662

62 0.14789565652608871
63 0.1415308378636837
64 0.15043034963309765
65 0.14304924104362726
66 0.13348282501101494
67 0.13463784009218216
68 0.135763649828732
69 0.13100494723767042
70 0.12752756290137768
71 0.13029645383358002
72 0.13424296490848064
73 0.12934745382517576
74 0.12773914448916912
75 0.1251143654808402
76 0.12306563556194305
77 0.12651420291513205
78 0.1253969483077526
79 0.11680148355662823
80 0.11503912881016731
81 0.11881735734641552
82 0.11773394234478474
83 0.11486559640616179
84 0.11538662761449814
85 0.11387072782963514
86 0.11068203207105398
87 0.1092904731631279
88 0.10933705233037472
89 0.10778840258717537
90 0.10781249962747097
91 0.10869042109698057
92 0.1075166305527091
93 0.10631470195949078
94 0.10407029557973146
95 0.10208761785179377
96 0.10173121653497219
97 0.10417866986244917
98 0.10265122447162867
99 0.09919378627091646
test error Net nh=500 7.10%% 71/1000
0 0.891412727534771
1 0.7435705587267876
2 0.6516740173101425
3 0.5863386578857899
4 0.545424204